In [1]:
url = 'https://www.encodeproject.org/search/?type=Experiment&control_type!=*&status=released&perturbed=false&assay_slims=DNA+accessibility&assay_title=DNase-seq&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&files.platform.term_name=Illumina+HiSeq+4000&files.run_type=paired-ended&replicates.library.nucleic_acid_term_name=DNA'
import requests
import os
import pandas as pd

encode_auth = ('3G5L4K3W',  'yqz7n22xndk5udyp')
HEADERS = {'accept': 'application/json'}


json_results = requests.get(url, headers=HEADERS, 
                            auth=encode_auth).json()

experiments = [results['accession'] for results in json_results['@graph']]

In [2]:
#json_results

In [5]:
import requests
import os
import pandas as pd
from tqdm import tqdm

        
encode_auth = ('3G5L4K3W',  'yqz7n22xndk5udyp')
HEADERS = {'accept': 'application/json'}


ChipReplicationQualityMetrics = {
    'reproducible_peaks', 'idr_cutoff', 'rescue_ratio', 
    'self_consistency_ratio', 'reproducibility'}

ChipPeakEnrichmentQualityMetrics = {
    'frip', 'min_size', '25_pct', '50_pct', '75_pct', 'max_size', 'mean'}

ChipAlignmentEnrichmentQualityMetrics = {
    'subsampled_reads', 'estimated_fragment_len', 'corr_estimated_fragment_len',
    'phantom_peak', 'corr_phantom_peak', 'argmin_corr', 'min_corr', 'NSC', 
    'RSC', 'auc', 'syn_auc', 'x_intercept', 'syn_x_intercept', 'elbow_pt', 
    'syn_elbow_pt', 'jsd', 'syn_jsd', 'pct_genome_enrich', 'diff_enrich', 
    'ch_div'}  

ChipLibraryQualityMetrics = {
    'unpaired_reads', 'paired_reads', 'unmapped_reads', 
    'unpaired_duplicate_reads', 'paired_duplicate_reads',
    'paired_optical_duplicate_reads', 'pct_duplicate_reads', 'total_fragments', 
    'distinct_fragments', 'positions_with_one_read', 'NRF', 'PBC1', 'PBC2'}

ChipAlignmentQualityMetrics = {
    'processing_stage', 'total_reads', 'total_reads_qc_failed', 
    'duplicate_reads', 'duplicate_reads_qc_failed', 'mapped_reads', 
    'mapped_reads_qc_failed', 'pct_mapped_reads', 'paired_reads', 
    'paired_reads_qc_failed', 'read1', 'read1_qc_failed', 'read2', 
    'read2_qc_failed', 'properly_paired_reads', 
    'properly_paired_reads_qc_failed', 'pct_properly_paired_reads', 
    'with_itself', 'with_itself_qc_failed', 'singletons', 
    'singletons_qc_failed', 'pct_singletons', 'diff_chroms', 
    'diff_chroms_qc_failed'}


rows = []
    

def get_metadata_for_experiment(exp_accession_id):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    accession_id = 'NNNN'

    search_url = "https://www.encodeproject.org/{}".format(
        exp_accession_id)
    json_results = requests.get(search_url, headers=HEADERS, 
                                auth=encode_auth).json()

    assembly = ''

    fastqs = []
    fastqs_run_type = []
    assay_type = []

    alignments = [] 
    alignments_download_urls = []
    alignments_md5sums = []
    alignments_qc_metrics = []

    unfiltered_alignments = []
    unfiltered_alignments_download_urls = []
    unfiltered_alignments_md5sums = []
    unfiltered_alignments_qc_metrics = []

    preferred_default_bed_narrowPeak = []
    preferred_default_bed_narrowPeak_download_urls = []
    preferred_default_bed_narrowPeak_md5sums = []
    preferred_default_bed_narrowPeak_qc_metrics = []
    
    number_of_peaks = []

    encode4_files = []
    
    try:
        json_results['analyses']
        
    except:
        print(f"une problem with for {exp_accession_id}")
        return
        
            
    for analysis_object in json_results['analyses']:
        if 'ENCODE4' in analysis_object['pipeline_award_rfas']:
            encode4_files = [f.split('/')[2] for f in \
                             analysis_object['files']]

    #print("ENCODE4 files", encode4_files)

    for file_dict in json_results['files']:
        if file_dict['file_type'] == 'fastq': 
            fastqs.append(file_dict['accession'])
            fastqs_run_type.append(file_dict['run_type'])
            continue

        if file_dict['accession'] not in encode4_files:
            #print("Skipping", file_dict['accession'])
            continue

        # fastq files dont have 'assembly' so we ^^^ check
        # that first
        # this check may not be necessary because we are 
        # checking for ENCODE4
        if file_dict['status'] == 'archived' or \
            file_dict['assembly'] == 'hg19':
            continue

        # this will get overwritten several times for each 
        # file, but it's ok 
        assembly = file_dict['assembly']

        if file_dict['file_type'] == 'bed narrowPeak' and \
            'preferred_default' in file_dict.keys() and \
            file_dict['preferred_default']:

            download_url = "https://www.encodeproject.org/" + \
               "files/{}/@@download/{}.bed.gz".format(
                   file_dict['accession'], 
                   file_dict['accession'])

            preferred_default_bed_narrowPeak.append(
                file_dict['accession'])
            preferred_default_bed_narrowPeak_download_urls.append(
                download_url)
            preferred_default_bed_narrowPeak_md5sums.append(
                file_dict['md5sum'])

            _qc_metrics = {}
            for qc_metrics in file_dict['quality_metrics']:
                if qc_metrics['@type'][0] == \
                    'ChipReplicationQualityMetric':
                    _qc_metrics['ChipReplicationQualityMetric'] = \
                        {k: qc_metrics[k] for k in \
                         qc_metrics.keys() & \
                         ChipReplicationQualityMetrics}

                if qc_metrics['@type'][0] == \
                    'ChipPeakEnrichmentQualityMetric':
                    _qc_metrics['ChipPeakEnrichmentQualityMetric'] = \
                        {k: qc_metrics[k] for k in \
                         qc_metrics.keys() & \
                         ChipPeakEnrichmentQualityMetrics}

            preferred_default_bed_narrowPeak_qc_metrics.append(
                _qc_metrics)
            
#             if 'reproducible_peaks' in file_dict['quality_metrics'][0]:
#                 number_of_peaks.append(file_dict['quality_metrics'][0]['reproducible_peaks'])
#             else:
#                 number_of_peaks.append(file_dict['quality_metrics'][1]['reproducible_peaks'])
            
            continue

        if file_dict['file_type'] == 'bam': 

            download_url = "https://www.encodeproject.org/" + \
                           "files/{}/@@download/{}.bam".format(
                               file_dict['accession'], 
                               file_dict['accession'])

            _qc_metrics = {}
            for qc_metrics in file_dict['quality_metrics']:
                if qc_metrics['@type'][0] == \
                    'ChipAlignmentEnrichmentQualityMetric':
                    _qc_metrics['ChipAlignmentEnrichmentQualityMetric'] = \
                        {k: qc_metrics[k] for k in \
                         qc_metrics.keys() & \
                         ChipAlignmentEnrichmentQualityMetrics}

                if qc_metrics['@type'][0] == \
                    'ChipLibraryQualityMetric':
                    _qc_metrics['ChipLibraryQualityMetric'] = \
                        {k: qc_metrics[k] for k in \
                         qc_metrics.keys() & \
                         ChipLibraryQualityMetrics}

                if qc_metrics['@type'][0] == \
                    'ChipAlignmentQualityMetric':
                    _qc_metrics['ChipAlignmentQualityMetric'] = \
                        {k: qc_metrics[k] for k in \
                         qc_metrics.keys() & \
                         ChipAlignmentQualityMetrics}

            assay_type=file_dict["assay_title"]
            if file_dict['output_type'] == 'unfiltered alignments':
                unfiltered_alignments.append(file_dict['accession'])  
                unfiltered_alignments_download_urls.append(download_url)
                unfiltered_alignments_md5sums.append(file_dict['md5sum'])
                unfiltered_alignments_qc_metrics.append(_qc_metrics)

            if file_dict['output_type'] == 'alignments':
                alignments.append(file_dict['accession'])  
                alignments_download_urls.append(download_url)
                alignments_md5sums.append(file_dict['md5sum'])
                alignments_qc_metrics.append(_qc_metrics)    

    row = [accession_id, assay_type, exp_accession_id, fastqs, fastqs_run_type,
           assembly, preferred_default_bed_narrowPeak, 
           preferred_default_bed_narrowPeak_download_urls, 
           preferred_default_bed_narrowPeak_md5sums, 
           preferred_default_bed_narrowPeak_qc_metrics,
           unfiltered_alignments, unfiltered_alignments_download_urls,
           unfiltered_alignments_md5sums, 
           unfiltered_alignments_qc_metrics, alignments, 
           alignments_download_urls, alignments_md5sums, 
           alignments_qc_metrics]

    #print(row)

    unique_run_types = set(fastqs_run_type)
    if len(unique_run_types) > 1:
        print(accession_id, exp_accession_id, "run_types", 
              len(unique_run_types), "****** MULTIPLE RUN TYPES ******")
        
    return (row)

In [6]:
import multiprocessing as mp

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(40)

# Step 2: `pool.apply` the `howmany_within_range()`
results = pool.map(get_metadata_for_experiment, experiments)

# Step 3: Don't forget to close
pool.close() 

In [7]:
filtered_results = [i for i in results if type(i)==type([])]

In [8]:
df = pd.DataFrame(filtered_results) 
columns = ['experiment_series','assay_type' ,'experiment', 'fastqs', 'fastqs_run_type', 
       'assembly', 'preferred_default_bed_narrowPeak', 
       'preferred_default_bed_narrowPeak_download_urls', 
       'preferred_default_bed_narrowPeak_md5sums',
       'preferred_default_bed_narrowPeak_qc_metrics', 
       'unfiltered_alignments', 'unfiltered_alignments_download_urls', 
       'unfiltered_alignments_md5sums', 'unfiltered_alignments_qc_metrics',
       'alignments', 'alignments_download_urls', 'alignments_md5sums', 
       'alignments_qc_metrics']
df.to_csv('metadata.tsv', 
           sep='\t', index=False, header=columns)

In [9]:
df = pd.read_csv('metadata.tsv', 
                            sep='\t')

In [10]:
alignments_urls = []
alignments_md5sums = []
unfiltered_alignments_urls = []
unfiltered_alignments_md5sums = []

for idx, row in df.iterrows():
    unfiltered_alignments_urls.extend(row['unfiltered_alignments_download_urls'])
    unfiltered_alignments_md5sums.extend(row['unfiltered_alignments_md5sums'])
    
    alignments_urls.extend(row['alignments_download_urls'])
    alignments_md5sums.extend(row['alignments_md5sums'])
    

alignments_urls_md5sums = ([list(a) for a in zip(alignments_urls, alignments_md5sums)])
alignments_urls_md5sums_df = pd.DataFrame(alignments_urls_md5sums)
alignments_urls_md5sums_df.to_csv('alignments_bams_urls_md5sums.txt', header=None, index=False, sep=' ')

unfiltered_alignments_urls_md5sums = ([list(a) for a in zip(unfiltered_alignments_urls, unfiltered_alignments_md5sums)])
unfiltered_alignments_urls_md5sums_df = pd.DataFrame(unfiltered_alignments_urls_md5sums)
unfiltered_alignments_urls_md5sums_df.to_csv('unfiltered_alignments_bams_urls_md5sums.txt', header=None, index=False, sep=' ')

# preferred default IDR thresholded peaks
preferred_default_urls = []
preferred_default_md5sums = []
for idx, row in df.iterrows():
    preferred_default_urls.extend(row['preferred_default_bed_narrowPeak_download_urls'])
    preferred_default_md5sums.extend(row['preferred_default_bed_narrowPeak_md5sums'])

preferred_default_urls_md5sums = ([list(a) for a in zip(preferred_default_urls, preferred_default_md5sums)])

preferred_default_urls_md5sums_df = pd.DataFrame(preferred_default_urls_md5sums)
preferred_default_urls_md5sums_df.to_csv('preferred_default_urls_md5sums.txt', header=None, index=False, sep=' ')